# Adding Back the Tides - CIOPS
An effort to make the daily files more accurate as they are currently lacking the tidal pumping that is so important to the flow of the Salish Sea

In [1]:
import xarray as xr
from pathlib import Path
import numpy as np
import datetime as dt

In [5]:
start = dt.datetime(2015,11,22) #start day of your run

#dates for each run
numdays = 15 #15 for all except the last run (14)
date_list = [start + dt.timedelta(days=x) for x in range(numdays)]
folderday = [start+ dt.timedelta(days=7),start+ dt.timedelta(days=7*2),start+ dt.timedelta(days=7*3)]
folderday = np.repeat(folderday,7)
folderday = folderday[:-6]

In [12]:
"{:%Y%m%d}00/BC12_1h_grid_U_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[0], date_list[0], date_list[0])

'2015112900/BC12_1h_grid_T_2D_20151122_20151122.nc'

In [16]:
path = Path("/ocean/mdunphy/CIOPSW-BC12/")

drop_vars = (
    "sbu", "tauuo", "time_counter_bounds", "time_instant_bounds", "uos",
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1h_grid_U_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[i], date_list[i], date_list[i]))) for i in range(len(date_list))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
ut_h = mydata['ubar']

In [23]:
drop_vars = (
    "sbv", "tauvo", "time_counter_bounds", "time_instant_bounds", "vos",
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1h_grid_V_2D_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[i], date_list[i], date_list[i]))) for i in range(len(date_list))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
vt_h = mydata['vbar']

In [25]:
# 2D hourly outputs are alraedy in bartropic form so you dont need to do any conversion!

In [24]:
# Now get the required data from the daily files
drop_vars = (
    "depthu_bounds", "nav_lat", "nav_lon", 'time_counter_bounds', 'time_instant',
    'time_instant_bounds', 
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1d_grid_U_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[i], date_list[i], date_list[i]))) for i in range(len(date_list))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
u_d = mydata['uo']

drop_vars = (
    "depthv_bounds", "nav_lat", "nav_lon", 'time_counter_bounds', 'time_instant',
    'time_instant_bounds', 
)

files = [sorted(path.glob("{:%Y%m%d}00/BC12_1d_grid_V_{:%Y%m%d}_{:%Y%m%d}.nc".format(folderday[i], date_list[i], date_list[i]))) for i in range(len(date_list))]

mydata = xr.open_mfdataset(files, drop_variables=drop_vars)
v_d = mydata['vo']

In [27]:
# we WILL need to do some conversions here, so get e3t from the mesh_mask file
mydata = xr.open_dataset("/ocean/mdunphy/mesh_mask.nc")
e3t = mydata['e3t_0']
e3t

<xarray.DataArray 'e3t_0' (t: 1, z: 50, y: 1021, x: 715)>
[36500750 values with dtype=float64]
Dimensions without coordinates: t, z, y, x

In [38]:
# convert e3t to e3u and to e3v
e3t_xshift = e3t.shift(x=-1,fill_value=0)
e3u = e3t_xshift+e3t
e3u = e3u*0.5
e3u = e3u.rename({'z': 'depthu'})

e3t_yshift = e3t.shift(y=-1,fill_value=0)
e3v = e3t_yshift+e3t
e3v = e3v*0.5
e3v = e3v.rename({'z': 'depthv'})

In [47]:
#calcuate bartropic component
ut_d = (u_d*e3u[]).sum(dim='depthu')/e3u[.sum(dim='depthu')

In [62]:
#subtract from u to get baroclinic component
uc_d = u_d-ut_d #does this work even though their ut_d lacks the depth dimension?

interpolate + resample uc_d to get it in an hourly format

In [63]:
uc_h_interp = uc_d.resample(time_counter="1H", loffset="30min").interpolate("linear")

In [64]:
u_new = ut_h  + uc_h_interp

In [66]:
u_new

<xarray.DataArray (time_counter: 25, y: 898, x: 398, depthu: 40)>
dask.array<add, shape=(25, 898, 398, 40), dtype=float32, chunksize=(13, 898, 398, 40), chunktype=numpy.ndarray>
Coordinates:
  * time_counter  (time_counter) datetime64[ns] 2019-02-28T12:30:00 ... 2019-...
    nav_lat       (y, x) float32 dask.array<chunksize=(898, 398), meta=np.ndarray>
    nav_lon       (y, x) float32 dask.array<chunksize=(898, 398), meta=np.ndarray>
  * depthu        (depthu) float32 0.5 1.5 2.5 3.5 ... 360.7 387.6 414.5 441.5
Dimensions without coordinates: y, x

In [ ]:
u_new.to_netcdf(str(path)+'u_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[-1]))

In [ ]:
#calcuate bartropic component
vt_d = (v*e3v).sum(dim='depthv')/e3v.sum(dim='depthv')

In [ ]:
#subtract from v to get baroclinic component
vc_d = v-vt_d 

In [ ]:
vc_d.load(scheduler="processes", num_workers=n)

In [ ]:
vc_h_interp = vc_d.resample(time_counter="1H", loffset="30min").interpolate("linear")

In [ ]:
v_new = vt_h  + vc_h_interp

In [ ]:
v_new.to_netcdf(str(path)+'v_new_{:%d%b%y}_{:%d%b%y}.nc'.format(date_list[0],date_list[-1]))